<a href="https://colab.research.google.com/github/florinabas/Neural-Pathways-Analysis/blob/main/Neuron_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 85.6 MB/s eta 0:00:00


In [ ]:
# Scripting NEURON basics tutorial - https://neuron.yale.edu/neuron/docs/scripting-neuron-basics

################################################################################
#Step 1: Import the neuron module into Python
import neuron
print(neuron.__version__)
from neuron import h, rxd
from neuron.units import ms, mV

################################################################################
#Step 2: Create a cell
soma = h.Section(name='soma')
# displays the topological structure of the entire model
# demonstrates that the soma has been created and has one segment (one dash is shown)
h.topology()

#The psection method - properties of the section
soma.psection()
#the soma is a cylinder with length 100 microns, diameter 500 microns, axial resistivity 35.4 ohm*cm, and specific membrance capacitance 1 μF/cm2.
#it is a dictionary, we can extract any properties we want using square brackets
soma.psection()['morphology']['L']
#All of these values can be individually accessed in more efficient ways, but psection provides an overview of the full properties of the section.
#simpler way
soma.L

#noticed that the default diameter is 500 μm - excessively large for mammalian neurons
#default value -  squid giant axons studied by Hodgkin and Huxley
# same for axial resistivity (soma.Ra) and temperature (h.celsius) - should all be adjusted for mammalian models

################################################################################
#Step 3: Set the cell's morphological properties
soma.L = 20
soma.diam = 20
#probe objects with Python’s built-in dir() function
dir(soma)
#tells us all of the Python methods and variables associated with the object
import textwrap
#print(textwrap.fill(', '.join(dir(h))))
help(soma.connect)

# # Biophysical mechanisms
#Thousands of additional mechanisms (for A-currents, etc) are available as MOD files as part of published model codes on ModelDB.
  # hh	- Hodgkin-Huxley sodium, potassium, and leakage channels.
  # pas -	Passive (“leak”) channel.
  # extracellular -	For simulating effects of nonzero extracellular potential - leaky patch clamps, or detailed propertes of the myelin sheath.

################################################################################
#Step 4: Insert ion channels
soma.insert('hh')
#HH channel kinetics based on the squid giant axon - other kinetics - by downloading them from online resources like ModelDB or by writing them yourself in NMODL or NeuroML

## Step 5: Sections and segments
#a NEURON Section is considered a piece of cable
#may be necessary to divide the cable into a number of segments where voltage varies linearly between centers of adjacent segments
#nseg - The number of segments within a section, divides the cable into equal-length parts - should be an odd number, (0.5) gives the middle segment.
#total ionic current across the segment membrane  - approx. area of the segment multiplied by the ionic current density at the center of the segment
#to access a part of the section, specify a value between 0 and 1 - 0 is typically the end closest to the soma and 1 is the distal end
#access sections by their name and segments by some location on the section.
  #Section: section
  #Segment: section(loc)
print("type(soma) = {}".format(type(soma)))
print("type(soma(0.5)) = {}".format(type(soma(0.5))))

#Accessing segment variables
# Segment variables follow the idiom:
  # section(loc).var
# And for mechanisms on the segment:
  # section(loc).mech.var

mech = soma(0.5).hh
print(dir(mech))
print(mech.gkbar)
print(soma(0.5).hh.gkbar)

# soma with HH channels and inside that we have gating variables like m, n, h
#soma(0.5).hh.m

################################################################################
## Step 5: Insert a stimulus
#insert a current clamp (an IClamp object) into the center of the soma to induce some membrane dynamics.
#An IClamp is a Point Process - point sources of current; when making a new PointProcess, you pass the segment to which it will bind.
iclamp = h.IClamp(soma(0.5))
#with the dir function, we can validate that iclamp is an object and contains some useful parameters
print([item for item in dir(iclamp) if not item.startswith('__')])

#key properties of a current clamp:
  #amp -- the amplitude (in nA),
  #delay -- the time the current clamp switches on (in ms)
  #dur -- how long (in ms) the current clamp stays on
iclamp.delay = 2
iclamp.dur = 0.1
iclamp.amp = 0.9

#Let's use psection to get a representation of the soma model:
soma.psection()

################################################################################
#Step 6: Set up recording variables
#The cell should be configured to run a simulation.
  #indicate which variables we wish to record
  #store in a NEURON Vector (h.Vector object)

#record
  #membrane potential - soma(0.5).v
  #corresponding time points (h.t)
#References to variables are available by preceding the last part of the variable name with a _ref_
v = h.Vector().record(soma(0.5)._ref_v)             # Membrane potential vector
t = h.Vector().record(h._ref_t)                     # Time stamp vector

################################################################################
#Step 7: Run the simulation
#By default, the NEURON h module provides the low level fadvance function for advancing one time step. For higher-level simulation control specification, we load NEURON's stdrun library:
h.load_file('stdrun.hoc')
#initialize our simulation such that our cell has a resting membrane potential of -65 mV:
# initialized to a resting membrane potential of -65 mV because that's the default reversal potential for the hh channels, the only channel (set) inserted in this model
h.finitialize(-65 * mV)
#continue the simulation from the current time (0) until 40 ms:
h.continuerun(10 * ms)
#we didn't need to specify the units here -- recall they were defined above in the from neuron.units import ms, mV -- as they are the defaults assumed by NEURON, but it is good practice to be explicitly clear

################################################################################
#Step 8: Plot the results
from bokeh.io import output_notebook
import bokeh.plotting as plt
output_notebook()

#Now we plot membrane potential vs time.
f = plt.figure(x_axis_label='t (ms)', y_axis_label='v (mV)')
f.line(t, v, line_width=2)
plt.show(f)

# # equivalent
#%matplotlib inline
#import matplotlib.pyplot as plt
#plt.figure()
#plt.plot(t, v)
#plt.xlabel('t (ms)')
#plt.ylabel('v (mV)')
#plt.show()

8.2.2

|-|       soma(0-1)

Help on built-in function connect:

connect(...) method of nrn.Section instance
    childSection.connect(parentSection, [parentX], [childEnd]) or
    childSection.connect(parentSegment, [childEnd])

type(soma) = <class 'nrn.Section'>
type(soma(0.5)) = <class 'nrn.Segment'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'el', 'gk', 'gkbar', 'gl', 'gna', 'gnabar', 'h', 'hinf', 'htau', 'il', 'is_ion', 'm', 'minf', 'mtau', 'n', 'name', 'ninf', 'ntau', 'segment']
0.036
0.036
['amp', 'baseattr', 'delay', 'dur', 'get_loc', 'get_segment', 'has_loc', 'hname', 'hocobjptr', 'i', 'loc', 'same']


In [ ]:
# BALL AND STICK MODEL PART 1 tutorial - https://neuron.yale.edu/neuron/docs/ball-and-stick-model-part-1
from neuron import h
from neuron.units import ms, mV
h.load_file('stdrun.hoc')
%matplotlib notebook

###########################################
#Defining the cell morphology
# create sections
#A ball-and-stick cell by definition consists of two parts: the soma (ball) and a dendrite (stick)
class BallAndStick:
    def __init__(self,gid):
        self._gid = gid #There is no way to tell the two somas apart and see which goes with which cell. To fix this, we'll pass in an identifier gid when we create the cell and have __repr__ incorporate that into the name:
        self._setup_morphology()
        self._setup_biophysics()
    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.all = [self.soma, self.dend] #We've added a new variable self.all which is a list of all the sections in the cell
        self.dend.connect(self.soma)  #connect sections -  not equivalent to attaching the soma to the dend, but dendrite begins where the soma ends - diff topology
        self.soma.L = self.soma.diam = 12.6157 # the chosen numbers  make the surface area approximately 500 μm2
        self.dend.L = 200
        self.dend.diam = 1
    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        #Hodgkin-Huxley (hh) kinetics in the soma  and specify param
        self.soma.insert('hh')
        for seg in self.soma:       #we loop over all segments in the soma, even though we only defined one segment - more general code                                              # <-- NEW
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003    # Leak conductance in S/cm2
            seg.hh.el = -54.3     # Reversal potential in mV
        # Insert passive current in the dendrite
        self.dend.insert('pas')
        for seg in self.dend:
            seg.pas.g = 0.001  # Passive conductance in S/cm2
            seg.pas.e = -65    # Leak reversal potential mV
    def __repr__(self):
         return 'BallAndStick[{}]'.format(self._gid)

#Any variables that describe properties of the cell must get stored as attributes of self. This is why we write self.soma instead of soma.
#Temporary variables need not be prefixed with self and will simply stop existing when the initialization function ends.
my_cell = BallAndStick(0)
#my_other_cell = BallAndStick(1)
#del my_other_cell
h.topology()
my_cell.soma(0.5).area()
#if not sure about the units for a given mechanism’s parameter, use units()
print(h.units('gnabar_hh'))
for sec in h.allsec():
    print('%s: %s' % (sec, ', '.join(sec.psection()['density_mechs'].keys())))

#A soma can have many dendrites attached to it
#any dendrite only begins at one specific location
# explicitly specify the connection location via
#self.dend.connect(self.soma(0.5)) #the dendrite was attached to the center of the soma

#if we're only interested in electrophysiology modeling -can  substitute a cylindrical soma with equal length and diameter for a spherical soma with the same diameter
 # different volume - substitution does not work if modeling diffusion or accumulation of ions.

## plot - not working
import matplotlib.pyplot as plt
h.PlotShape(False).plot(plt)
ps = h.PlotShape(True)
ps.show(0)

##########################3
#simulation - inject a current pulse into the distal end of the dendrite
#starting 5 ms after the simulation starts, with a duration of 1 m and an amplitude of 0.1 nA.
#define and position the current clamp object:
stim = h.IClamp(my_cell.dend(1))
# check the segment the current clamp is inserted into
stim.get_segment()
# if we forget what the names of the attributes are, we can check the dir
print(', '.join(item for item in dir(stim) if not item.startswith('__')))
stim.delay = 5
stim.dur = 1
stim.amp = 0.1

#recording the membrane potential at the center of the soma and the time in two NEURON Vectors
soma_v = h.Vector().record(my_cell.soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

#run simulation
h.finitialize(-65 * mV) #initialize membrane potential everywhere to -65 mV
h.continuerun(25 * ms) #run until time 25 ms

#To plot the dendrite potential, we need to record it in a NEURON Vector
dend_v = h.Vector().record(my_cell.dend(0.5)._ref_v)

#plot results
from bokeh.io import output_notebook
import bokeh.plotting as plt
output_notebook()
#plot membrane potential vs time
#set of simulations, plotted in the same figure, where we vary the amplitude of the current
f = plt.figure(x_axis_label='t (ms)', y_axis_label='v (mV)')
amps = [0.075 * i for i in range(1, 5)]
colors = ['green', 'blue', 'red', 'black']
for amp, color in zip(amps, colors):
    stim.amp = amp
    h.finitialize(-65 * mV)
    h.continuerun(25 * ms)
    f.line(t, list(soma_v), line_width=2, legend_label='amp=%g' % amp, color=color)
    # to add dendrites - only difference is one additional call to the line method to plot the dend_v using a dashed line without a separate legend entry:
    f.line(t, list(dend_v), line_width=2, line_dash='dashed', color=color)
plt.show(f)

#when the soma membrane potential is sufficiently low, it is possible for the dendrite to be more depolarized than the soma, but the peak membrane potential in the leaky dendrite is significantly reduced from the peak in the soma during an action potential.
#This is because in our model all the voltage-gated channels that would depolarize the cell are only present in the soma.)


In [ ]:
# run all of our simulations for both nseg = 1 (the default) and for nseg = 101
f = plt.figure(x_axis_label='t (ms)', y_axis_label='v (mV)')
amps = [0.075 * i for i in range(1, 5)]
colors = ['green', 'blue', 'red', 'black']
for amp, color in zip(amps, colors):
    stim.amp = amp
    for my_cell.dend.nseg, width in [(1, 2), (101, 1)]:
        h.finitialize(-65)
        h.continuerun(25)
        f.line(t, list(soma_v), line_width=width, legend_label='amp=%g',color=color)#% amp if my_cell.dend.nseg == 1 else None
        f.line(t, list(dend_v), line_width=width, line_dash='dashed',color=color)
plt.show(f)

#with the high-resolution simulation that the soma peaks should be reduced and delayed and the dendrite peaks increased relative to what was seen in the nseg=1 case.

In [ ]:
# Reaction Diffusion tutorial - https://neuron.yale.edu/neuron/docs/reaction-diffusion
from neuron import rxd

#Proteins, ions, etc... in a cell perform signalling functions by moving, reacting with other molecules, or both.
# In some cases, this movement is by active transport processes, which we do not consider here.
# If all movement is due to diffusion (wherein a molecule moves randomly), then such systems are known as reaction-diffusion systems.
#answers to three questions:

 #(1) Where do the dynamics occur
#electrophysiology simulations -only relevant domains - plasma membrane or the volumes immediately adjacent - regions responsible for generating the action potential
#Cell biology models - dynamics spanning a more varied set of locations - the endoplasmic reticulum (ER), mitochondria, and nuclear envelop often play key roles.
#rxd.Region class is used to describe the domain:
r = rxd.Region(sections, nrn_region=None, geometry=None)
#r = rxd.Region(h.allsec()) # all sections
#r = rxd.Region([soma, apical1, apical2]) # just on few sections
#r = rxd.Region(h.allsec(), nrn_region='i') # domain on the immediate interior of the membrane
#for the region just outside the membrane (the Frankenhaeuser-Hodgkin space) use nrn_region='o'
# For full 3D extracellular diffusion, define the region with rxd.Extracellular
#Many alternative geometries are available, including: rxd.membrane, rxd.inside (this is the default), rxd.Shell (used in the radial diffusion example; coming soon), rxd.FractionalVolume (used in the calcium wave example), rxd.FixedCrossSection, and rxd.FixedPerimeter

 #(2) Who are the actors
#chemical species (proteins, ions), State variables (such as a gating variable), parameters that vary throughout the domain
#described using the rxd.Species class
s = rxd.Species(regions=None, d=0, name=None, charge=0, initial=None, atolscale=1)
#although we also provide rxd.State and rxd.Parameter for the second and third case
#The regions parameter is mandatory and is either a single rxd.Region or an iterable of them
#Set d= to the diffusion coefficient for your species, if any
#Specify an option for the name= keyword argument to allow these state variables to map to the NEURON/NMODL range variables if the region's nrn_region is 'i' or 'o'
#Specify initial conditions via the initial= keyword argument. Set that to either a constant or a function that takes a node
#units of concentration are assumed to be in mM.
#use atolscale to indicate that the tolerance should be scaled for the corresponding variable

  #(3) How do they interact?
#Species interact via one or more chemical reactions
#r = rxd.Reaction(lhs, rhs, rate_f, rate_b=None, regions=None, custom_dynamics=False)
# lhs and rhs describe the reaction scheme and are expressed using arithmetic sums of integer multiples of a Species
#cacl2_reaction = rxd.Reaction(ca + 2 * cl, cacl2, kf)#an irreversible reaction to form calcium chloride
#ca and cl are rxd.Species instances and kf is the reaction rate
#Since custom_dynamics was not specified this is a mass-action reaction
#While we can sometimes ignore the reverse reactions due to them having a high energy barrier, the laws of physics imply that all reactions are in fact reversible
#cacl2_reaction = rxd.Reaction(ca + 2 * cl, cacl2, kf, kb)
#While mass-action works well for elementary reactions, this is often impractical for modeling intracellular dynamics where there are potentially many elementary reactions driving the observable reaction. In this case, we often turn to phenomenological models, such as Michelis-Menten kinetics or the Hill equation.
#To indicate these in NEURON, set custom_dynamics=True and specify the forward and backward rates as the corresponding formula
#Note that using Michaelis-Menten kinetics for enzymatic reactions is only appropriate under certain conditions, such as that the concentration of enzyme is low relative to the concentration of the substrate.
#rxd.Reaction describes a single molecular reaction. That is, if the left hand side involves aA+bB+cC
# then a , b , and c  are the stoichiometry coefficients. In particular, this means that a reaction of 2 ca + 4 cl goes to 2 cacl2 is not equivalent to a reaction where ca + 2 cl goes to cacl2, as the first requires four chloride and two calcium molecules to come together simulatenously before any reaction occurs, which would cause the [cl]
# to be raised to the fourth power instead of the second power.
#Reactions conserve mass. This property is especially important for stochastic simulations. Often, however, one might wish to model a source or a sink (e.g. protein degradation), or describe the dynamics of a state variable that is not a concentration. In this case, use the non-conservative rxd.Rate.
r = rxd.Rate(species, rate, regions=None, membrane_flux=False)
#The previous methods assume that all the reactants exist in the same region. (If they are not all present in a given location, then the reaction does not occur there.) Pumps and channels can allow material to move between compartments. These are implemented in NEURON using an rxd.MultiCompartmentReaction and using square brackets to specify regions. The rate is proportional to the area of the membrane between the regions, which must be specified. e.g. if ca is a species present on the rxd.Region objects er and cyt then a leak compartment across the ER membrane might be implemented as:
leak = rxd.MultiCompartmentReaction(ca[er], ca[cyt], gleak, gleak, membrane=cyt_er_membrane)

In [ ]:
# CNS turorial NEURON and NetPyNE - https://www.youtube.com/watch?v=1nIyGfxgdfI

from neuron import h
from neuron.units import mV, ms, um #unit specifications
from matplotlib import cm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'

h.load_file('stdrun.hoc')
soma=h.Section(name="soma")
soma.L=soma.diam=10*um
h.hh.insert(soma) #add HH ion channels
# give it a current clamp
ic=h.IClamp(soma(0.5))#put in the center of soma
ic.delay=1*ms
ic.dur=0.1*ms
ic.amp=0.1
# need to record it
t=h.Vector().record(h._ref_t)
v=h.Vector().record(soma(0.5)._ref_v)
#to run the simulation we need to initialize to some membrane potential
h.finitialize(-65*mV)
h.continuerun(10*ms)
# can look at all the voltages
list(v) # looks like it s not enough
go.Figure(data=go.Scatter(x=t,y=v)).show()

In [ ]:
#was not enough, increase
ic.amp=1 #nA
h.finitialize(-65*mV)
h.continuerun(10*ms)
go.Figure(data=go.Scatter(x=t,y=v)).show()

In [ ]:
fig=go.Figure()
for ic.amp in [0.1,0.2,0.3,0.4]:
  h.finitialize(-65*mV)
  h.continuerun(10*ms)
  fig.add_trace(go.Scatter(x=t,y=v, name=f"amp={ic.amp}"))
fig.show()

In [ ]:
soma(0.5).hh.m #gating variables m, n and h that can be recored

0.022646381209187886

In [ ]:
m=h.Vector().record(soma(0.5).hh._ref_m)
h.finitialize(-65*mV)
h.continuerun(10*ms)
go.Figure(data=go.Scatter(x=t,y=m)).show()


In [ ]:
#NeuroMorpho.Org – standardized morphology file
#Loading morphologies
from neuron import h
h.load_file('import3d.hoc')
cell=h.Import3d_SWC_read()
cell.input('filename.swc')
i3d=h.Import3d_GUI(cell,False)
i3d.instantiate(None)

In [ ]:
#Reaction Diffusion & extracellular - Newton tutorial

#action potentials arise from ions moving across the membrane
#Na current raise membrane potential and K current lower it
#what s a current - charges that are carried by the Na K ions



